# ⚡ Dask: Conquering MASSIVE Arrays ⚡

## 🔥 PREPARE FOR COMPUTATIONAL POWER 🔥

**We're about to process a 30,000 × 30,000 digital elevation model (~7GB of data)**

- 🗻 **900 MILLION elevation points** - larger than most regional DEMs
- 🧮 **Complex terrain analysis** - gradients, aspects, curvature at massive scale
- 📊 **Multi-scale visualization** - from satellite view to hiking-trail detail
- 💾 **Out-of-core processing** - dataset larger than typical RAM
- ⚡ **Parallel execution** - all CPU cores working simultaneously

**System Requirements:** 4GB+ RAM, 2+ CPU cores, 15GB disk space

In [ ]:
!pip install dask jupyter-server-proxy dask-image

In [ ]:
import dask
import dask.array as da
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
import time
import psutil
import os
from dask.distributed import Client
import warnings
from dask.distributed import Client, LocalCluster
warnings.filterwarnings('ignore')

# Enhanced plotting
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 10

print(f"🚀 System Specs:")
print(f"   RAM: {psutil.virtual_memory().total / 1024**3:.1f} GB")
print(f"   CPU Cores: {psutil.cpu_count()}")
print(f"   Dask Version: {dask.__version__}")

## ⚙️ Unleashing Parallel Power

In [ ]:
# Configure Dask for maximum performance
n_workers = min(6, psutil.cpu_count())
memory_per_worker = max(1, int(psutil.virtual_memory().total / (n_workers * 1024**3) * 0.8))
memory_limit = f'{memory_per_worker}GB'

client = Client(
    n_workers=n_workers,
    threads_per_worker=2,
    memory_limit=memory_limit,
    dashboard_address=':8787',
    silence_logs=False
)

print(f"🔥 Dask Cluster Online:")
print(f"   Workers: {n_workers}")
print(f"   Total Threads: {n_workers * 2}")
print(f"   Memory per Worker: {memory_limit}")
print(f"   Dashboard: {client.dashboard_link}")
print(f"\n⚡ Ready to process multi-GB datasets! ⚡")

## 🗻 Generating Realistic Massive Terrain

Creating a **30,000 × 30,000** realistic digital elevation model using multi-octave noise.
This represents a ~300km × 300km area at 10m resolution - **larger than Belgium!**

In [ ]:
# Massive array configuration
ARRAY_SIZE = (30000, 30000)  # 900 million points!
CHUNK_SIZE = (1500, 1500)   # 2.25M points per chunk = ~18MB
PIXEL_SIZE = 10.0           # 10 meters per pixel
EXTENT_KM = (ARRAY_SIZE[0] * PIXEL_SIZE) / 1000  # Total extent in km

print(f"🗻 Creating MASSIVE Terrain:")
print(f"   Grid Size: {ARRAY_SIZE[0]:,} × {ARRAY_SIZE[1]:,} = {np.prod(ARRAY_SIZE)/1e6:.0f} million points")
print(f"   Real-world Size: {EXTENT_KM:.0f}km × {EXTENT_KM:.0f}km")
print(f"   Data Volume: ~{np.prod(ARRAY_SIZE) * 8 / 1024**3:.1f} GB")
print(f"   Chunks: {ARRAY_SIZE[0]//CHUNK_SIZE[0]} × {ARRAY_SIZE[1]//CHUNK_SIZE[1]} = {(ARRAY_SIZE[0]//CHUNK_SIZE[0]) * (ARRAY_SIZE[1]//CHUNK_SIZE[1])} chunks")

def generate_realistic_terrain(shape, chunks, pixel_size=10.0):
    """Generate ultra-realistic terrain using multi-scale features.
    
    Creates terrain with:
    - Mountain ranges (large scale structure)
    - Valley networks (medium scale drainage)
    - Surface roughness (small scale detail)
    - Realistic elevation distribution
    """
    print("   🏔️  Generating mountain ranges...")
    
    # Create coordinate grids (normalized 0-1)
    x = da.linspace(0, 1, shape[1], chunks=chunks[1])
    y = da.linspace(0, 1, shape[0], chunks=chunks[0])
    X, Y = da.meshgrid(x, y, indexing='ij')
    
    print("   🏞️  Adding large-scale topography...")
    # Large scale: Continental features (wavelength ~100km)
    large_scale = 2000 * da.sin(X * 4 * np.pi) * da.cos(Y * 3 * np.pi)
    large_scale += 1500 * da.cos(X * 6 * np.pi + np.pi/4) * da.sin(Y * 4 * np.pi)
    
    print("   ⛰️  Creating mountain ranges...")
    # Mountain ranges: Gaussian peaks
    range1 = 3000 * da.exp(-((X - 0.3)**2 + (Y - 0.7)**2) / 0.05)
    range2 = 2500 * da.exp(-((X - 0.7)**2 + (Y - 0.3)**2) / 0.03)
    range3 = 2000 * da.exp(-((X - 0.5)**2 + (Y - 0.5)**2) / 0.08)
    
    # Ridge lines
    ridge1 = 1000 * da.exp(-da.minimum((Y - 0.2 - 0.3*X)**2 / 0.001, 5))
    ridge2 = 800 * da.exp(-da.minimum((Y - 0.8 + 0.2*X)**2 / 0.001, 5))
    
    print("   🌊  Adding medium-scale features...")
    # Medium scale: Valley networks (wavelength ~10km)
    medium_scale = 800 * da.sin(X * 20 * np.pi) * da.cos(Y * 18 * np.pi)
    medium_scale += 600 * da.cos(X * 25 * np.pi) * da.sin(Y * 22 * np.pi)
    
    print("   🏕️  Adding fine-scale detail...")
    # Small scale: Local topography (wavelength ~1km)
    small_scale = 200 * da.sin(X * 100 * np.pi) * da.cos(Y * 95 * np.pi)
    small_scale += 150 * da.cos(X * 120 * np.pi) * da.sin(Y * 110 * np.pi)
    
    print("   🌿  Adding surface roughness...")
    # Surface roughness: Random variations
    roughness = da.random.normal(0, 25, shape, chunks=chunks)
    
    print("   🏗️  Assembling final terrain...")
    # Combine all scales
    elevation = (large_scale + range1 + range2 + range3 + ridge1 + ridge2 + 
                medium_scale + small_scale + roughness)
    
    # Add base elevation and ensure no negative values
    elevation = da.maximum(elevation + 500, 0)
    
    return elevation

# Generate the massive terrain
print("\n🚀 GENERATING MASSIVE TERRAIN...")
start_time = time.time()

massive_terrain = generate_realistic_terrain(ARRAY_SIZE, CHUNK_SIZE, PIXEL_SIZE)

generation_time = time.time() - start_time
print(f"\n✅ Terrain computation graph built in {generation_time:.3f}s")
print(f"📊 Array Details: {massive_terrain}")
print(f"💾 Memory footprint: {massive_terrain.nbytes / 1024**3:.1f} GB (when computed)")
print(f"\n⚡ Ready for MASSIVE parallel processing! ⚡")

## 🧮 Advanced Terrain Analysis at Scale

Computing **slope, aspect, and curvature** across 900 million elevation points using proper finite differences with boundary handling.

In [ ]:
def compute_advanced_terrain_metrics(elevation, pixel_size=10.0):
    """Compute comprehensive terrain metrics using proper finite differences.
    
    Returns slope, aspect, curvature using map_overlap for correct boundary handling.
    """
    
    def terrain_analysis_block(block):
        """Analyze terrain metrics for a single block with proper boundary handling."""
        # Compute gradients using central differences
        gy, gx = np.gradient(block, pixel_size, pixel_size)
        
        # Slope (magnitude of gradient vector)
        slope = np.sqrt(gx**2 + gy**2)
        
        # Aspect (direction of steepest descent)
        # Use proper downslope direction convention
        aspect = np.arctan2(-gy, -gx)  # Negative for downslope
        aspect = np.where(aspect < 0, aspect + 2*np.pi, aspect)  # Convert to 0-2π
        
        # Second derivatives for curvature
        gyy, gyx = np.gradient(gy, pixel_size, pixel_size)
        gxy, gxx = np.gradient(gx, pixel_size, pixel_size)
        
        # Mean curvature (simplified)
        mean_curvature = -(gxx + gyy) / 2.0
        
        # Stack all results
        return np.stack([slope, aspect, mean_curvature], axis=0)
    
    print("   🔬 Computing terrain derivatives with boundary handling...")
    # Apply to all chunks with overlapping boundaries to avoid edge artifacts
    results = da.map_overlap(
        terrain_analysis_block,
        elevation,
        depth=2,  # 2-pixel overlap for gradient computation
        boundary='reflect',  # Reflect at boundaries
        dtype=np.float64,
        new_axis=0,  # Add new axis for different metrics
        chunks=(3,) + elevation.chunks  # 3 metrics + original chunk structure
    )
    
    # Split into individual arrays
    slope = results[0]
    aspect = results[1] 
    curvature = results[2]
    
    return {
        'slope': slope,
        'aspect': aspect,
        'curvature': curvature
    }

print("🧮 COMPUTING ADVANCED TERRAIN ANALYSIS...")
print(f"   Processing {np.prod(ARRAY_SIZE)/1e6:.0f} million elevation points")
print(f"   Computing: Slope, Aspect, Curvature with proper boundary handling")

analysis_start = time.time()

# Compute all terrain metrics
terrain_metrics = compute_advanced_terrain_metrics(massive_terrain, PIXEL_SIZE)

setup_time = time.time() - analysis_start
print(f"\n✅ Analysis computation graph built in {setup_time:.3f}s")

# Show what we've created
for name, array in terrain_metrics.items():
    print(f"   📊 {name}: {array.shape} @ {array.dtype}")

print(f"\n📈 Total data volume: {sum(arr.nbytes for arr in terrain_metrics.values()) / 1024**3:.1f} GB")
print(f"⏱️  Ready for parallel execution!")

## ⚡ EXECUTING MASSIVE PARALLEL COMPUTATION

Time to **unleash the full power** of parallel processing on our massive dataset!

In [ ]:
# === Massive Dask Terrain Analysis — drop-in ready ===
import os, time
import numpy as np
import psutil
import dask.array as da
from dask.distributed import Client, LocalCluster

# -----------------------------
# Config
# -----------------------------
ARRAY_SIZE = (12000, 12000)     # ~144M pts
CHUNK_SIZE = (1500, 1500)       # tune for RAM/cores
CELL_SIZE  = 1.0                # meters
PERCENTILE_METHOD = "tdigest"   # "tdigest" or "dask" preferred; auto-fallback handled
SEED = 42

# -----------------------------
# Dask cluster
# -----------------------------
cluster = LocalCluster(
    threads_per_worker=2,
    n_workers=max(1, os.cpu_count() // 4),
    processes=True,
    dashboard_address=None,
)
client = Client(cluster)

# -----------------------------
# Gaussian helper (dask-image if available; else SciPy via map_overlap)
# -----------------------------
try:
    from dask_image.ndfilters import gaussian_filter as _da_gaussian
    def dask_gaussian(x, sigma):
        y = _da_gaussian(x, sigma=sigma)
        return y.astype(x.dtype, copy=False)
except Exception:
    import scipy.ndimage as ndi
    def dask_gaussian(x, sigma):
        depth = int(3 * sigma)
        return da.map_overlap(
            ndi.gaussian_filter, x,
            sigma=sigma, depth=depth, boundary='reflect', dtype=x.dtype,
        )

# RNG (Dask uses size=..., no dtype kw)
rs = da.random.RandomState(SEED)
def smooth_field(shape, chunks, scale):
    return rs.random(size=shape, chunks=chunks).astype(np.float32) * scale

# -----------------------------
# Build terrain (ensure multi-chunk for tdigest)
# -----------------------------
massive_terrain = (
    dask_gaussian(smooth_field(ARRAY_SIZE, CHUNK_SIZE, 300.0), sigma=8)
  + dask_gaussian(smooth_field(ARRAY_SIZE, CHUNK_SIZE, 150.0), sigma=16)
  + dask_gaussian(smooth_field(ARRAY_SIZE, CHUNK_SIZE,  50.0), sigma=32)
).astype(np.float32).rechunk(CHUNK_SIZE)

# -----------------------------
# Terrain metrics
# -----------------------------
gy, gx = da.gradient(massive_terrain, CELL_SIZE)   # returns d/dy, d/dx
slope = da.sqrt(gx**2 + gy**2)
aspect = da.arctan2(-gy, -gx)                      # downslope aspect
dgy_dy, dgy_dx = da.gradient(gy, CELL_SIZE)
dgx_dy, dgx_dx = da.gradient(gx, CELL_SIZE)
curvature = (dgx_dx + dgy_dy)                      # Laplacian-like
terrain_metrics = {"slope": slope, "aspect": aspect, "curvature": curvature}

# -----------------------------
# Robust global percentiles helper
# -----------------------------
def global_percentiles(a, qs=(25, 50, 75), method="midpoint"):
    a1 = a.ravel()
    # Count total chunks; if it's 1, NumPy will be used -> no "tdigest"
    n_chunks = 1
    for c in a1.chunks:
        n_chunks *= len(c)
    use_method = method if (n_chunks > 1 and method in ("tdigest", "dask")) else "linear"
    return da.percentile(a1, qs, method=use_method)

# -----------------------------
# Pretty header + system before
# -----------------------------
print("⚡ LAUNCHING MASSIVE PARALLEL COMPUTATION ⚡")
print("━" * 60)
memory_before = psutil.virtual_memory()
cpu_before = psutil.cpu_percent(interval=1)
print(f"🖥️  System Status Before:")
print(f"   RAM: {memory_before.used/1024**3:.1f}/{memory_before.total/1024**3:.1f} GB ({memory_before.percent:.1f}%)")
print(f"   CPU: {cpu_before:.1f}%")

# -----------------------------
# Phase 1: elevation stats
# -----------------------------
print(f"\n🔄 Phase 1: Computing elevation statistics...")
print(f"\n🔄 Phase 1: Computing elevation statistics...")
stats_start = time.time()

def safe_global_percentiles(arr, qs=(25, 50, 75), preferred="tdigest"):
    """Use Dask percentiles; if NumPy gets invoked (single-chunk) and pukes on 'tdigest',
    auto-fallback to a NumPy-supported method."""
    a1 = arr.rechunk(CHUNK_SIZE).ravel()  # ensure multi-chunk before ravel (helps Dask paths)
    try:
        p = da.percentile(a1, qs, method=preferred)
        # try computing alone first to trigger early if it will fall back to NumPy
        (p_vals,) = da.compute(p)
    except Exception:
        # fall back to a NumPy-compatible method
        p = da.percentile(a1, qs, method="linear")
        (p_vals,) = da.compute(p)
    return p_vals  # numpy array of quantiles

q1, q2, q3 = safe_global_percentiles(massive_terrain, (25, 50, 75), preferred=PERCENTILE_METHOD)

elev_stats = da.compute(
    massive_terrain.min(),
    massive_terrain.max(),
    massive_terrain.mean(),
    massive_terrain.std(),
)
stats_time = time.time() - stats_start

min_elev, max_elev, mean_elev, std_elev = elev_stats
print(f"✅ Elevation analysis completed in {stats_time:.2f}s")
print(f"📊 Elevation Statistics:")
print(f"   Range: {min_elev:.1f} - {max_elev:.1f} m ({max_elev-min_elev:.1f} m relief)")
print(f"   Mean ± Std: {mean_elev:.1f} ± {std_elev:.1f} m")
print(f"   Quartiles: Q1={q1:.0f} Q2={q2:.0f} Q3={q3:.0f}")

stats_time = time.time() - stats_start
min_elev, max_elev, mean_elev, std_elev = elev_stats
print(f"✅ Elevation analysis completed in {stats_time:.2f}s")
print(f"📊 Elevation Statistics:")
print(f"   Range: {min_elev:.1f} - {max_elev:.1f} m ({max_elev-min_elev:.1f} m relief)")
print(f"   Mean ± Std: {mean_elev:.1f} ± {std_elev:.1f} m")
print(f"   Quartiles: Q1={q1:.0f} Q2={q2:.0f} Q3={q3:.0f}")

# -----------------------------
# Phase 2: terrain metric stats
# -----------------------------
n_workers = len(client.scheduler_info()['workers'])
print(f"\n🔥 Phase 2: MASSIVE terrain analysis computation...")
print(f"   🎯 Target: {len(terrain_metrics)} metrics × {np.prod(ARRAY_SIZE)/1e6:.0f} million points each")
print(f"   ⚡ Workers: {n_workers} parallel workers")
terrain_start = time.time()

slope_stats = da.compute(
    terrain_metrics['slope'].mean(),
    terrain_metrics['slope'].max(),
    terrain_metrics['slope'].std(),
    (terrain_metrics['slope'] > np.tan(np.deg2rad(45.0))).sum()
)
aspect_stats = da.compute(
    terrain_metrics['aspect'].mean(),
    terrain_metrics['aspect'].std()
)
curvature_stats = da.compute(
    terrain_metrics['curvature'].mean(),
    terrain_metrics['curvature'].std(),
    (terrain_metrics['curvature'] > 0).sum(),
    (terrain_metrics['curvature'] < 0).sum()
)
terrain_time = time.time() - terrain_start

slope_mean, slope_max, slope_std, steep_count = slope_stats
aspect_mean, aspect_std = aspect_stats
curv_mean, curv_std, convex_count, concave_count = curvature_stats

# -----------------------------
# Summary
# -----------------------------
print(f"\n🎉 MASSIVE COMPUTATION COMPLETED! 🎉")
print(f"⏱️  Total computation time: {terrain_time:.2f}s")
bytes_processed = 4 * np.prod(ARRAY_SIZE) * 8  # heuristic
print(f"🚀 Processing rate: {(bytes_processed / 1024**3) / terrain_time:.2f} GB/s")
memory_after = psutil.virtual_memory()
print(f"💾 Memory usage: {memory_after.used/1024**3:.1f} GB (+{(memory_after.used-memory_before.used)/1024**3:.1f} GB)")

print(f"\n📈 TERRAIN ANALYSIS RESULTS:")
print("━" * 40)
print(f"🏔️  Slope Analysis:")
print(f"   Mean slope: {slope_mean:.3f} ({np.degrees(np.arctan(slope_mean)):.1f}°)")
print(f"   Max slope:  {slope_max:.3f} ({np.degrees(np.arctan(slope_max)):.1f}°)")
print(f"   Steep areas (>45°): {steep_count:,} points ({steep_count/np.prod(ARRAY_SIZE)*100:.1f}%)")

print(f"\n🧭 Aspect Analysis:")
print(f"   Mean aspect: {aspect_mean:.3f} rad ({np.degrees(aspect_mean):.1f}°)")
print(f"   Aspect variability: {aspect_std:.3f} rad")

print(f"\n🌊 Curvature Analysis:")
print(f"   Mean curvature: {curv_mean:.2e} m⁻¹")
print(f"   Convex areas: {convex_count:,} points ({convex_count/np.prod(ARRAY_SIZE)*100:.1f}%)")
print(f"   Concave areas: {concave_count:,} points ({concave_count/np.prod(ARRAY_SIZE)*100:.1f}%)")

print(f"\n🏆 ACHIEVEMENT: Processed {np.prod(ARRAY_SIZE)/1e6:.0f} million points! 🏆")
# === end ===


## 🔍 MULTI-SCALE VISUALIZATION SPECTACULAR

**The grand finale!** Multi-level zoom visualization showing the **full power** of our massive computation.

From **satellite overview** (300km view) to **hiking detail** (5km view) - all from the same massive dataset!

In [ ]:
import time, numpy as np, dask.array as da
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
from matplotlib.patches import Rectangle
from matplotlib.ticker import MaxNLocator

def create_multiscale_visualization(elevation, slope, aspect, curvature, array_size, pixel_size):
    """Multi-scale terrain viz with correct nested zooms, global km axes, and next-zoom red boxes."""
    print("🎨 CREATING SPECTACULAR MULTI-SCALE VISUALIZATION")
    print("━" * 55)

    ps_km = pixel_size / 1000.0
    full_x_km = array_size[1] * ps_km
    full_y_km = array_size[0] * ps_km
    EXTENT_KM = min(full_x_km, full_y_km)

    # km -> pixels (rounded)
    def km_to_px(km):
        return int(round(km / ps_km))

    # Center everything on the domain center to guarantee nesting
    cy, cx = array_size[0] // 2, array_size[1] // 2

    # Desired zoom sizes in km (Full, 100, 30, 2)
    zoom_defs = [
        dict(name='🗺️  REGIONAL VIEW', size_km=None, downsample=30,
             desc=lambda: f'{EXTENT_KM:.0f}km × {EXTENT_KM:.0f}km - Full Dataset'),
        # dict(name='🗺️  REGIONAL VIEW',  size_km=100.0, downsample=8,
        #      desc=lambda: '100km × 100km - Regional'),
        dict(name='🏔️  VALLEY VIEW',    size_km=30.0,  downsample=3,
             desc=lambda: '30km × 30km - Valley'),
        dict(name='🥾 HIKING DETAIL',    size_km=2.0,   downsample=1,
             desc=lambda: '2km × 2km - Local'),
    ]

    # Compute pixel windows, nested & clipped to domain
    zoom_levels = []
    prev_window = (0, array_size[0], 0, array_size[1])  # y0,y1,x0,x1 for Full
    for i, z in enumerate(zoom_defs):
        if z['size_km'] is None:
            y0, y1, x0, x1 = prev_window  # full domain
        else:
            half = km_to_px(z['size_km']) // 2
            # start from global center
            y0 = cy - half; y1 = cy + half
            x0 = cx - half; x1 = cx + half
            # clip to domain
            y0 = max(0, y0); y1 = min(array_size[0], y1)
            x0 = max(0, x0); x1 = min(array_size[1], x1)
            # ensure nesting inside previous window
            py0, py1, px0, px1 = prev_window
            if y0 < py0: y0 = py0
            if y1 > py1: y1 = py1
            if x0 < px0: x0 = px0
            if x1 > px1: x1 = px1
            # if still too small due to borders, shrink as needed (keeps nesting)
            y0 = int(y0); y1 = int(y1); x0 = int(x0); x1 = int(x1)
        window = (int(y0), int(y1), int(x0), int(x1))
        zoom_levels.append(dict(
            name=z['name'], description=z['desc'](),
            downsample=z['downsample'], window=window
        ))
        prev_window = window  # next must be inside this

    fig = plt.figure(figsize=(20, 16))
    print(f"📊 Extracting data for {len(zoom_levels)} zoom levels...")

    extracted = []
    for lvl in zoom_levels:
        y0, y1, x0, x1 = lvl['window']
        ds = lvl['downsample']
        t0 = time.time()

        elev_subset = elevation[y0:y1, x0:x1][::ds, ::ds]
        slope_subset = slope[y0:y1, x0:x1][::ds, ::ds]
        aspect_subset = aspect[y0:y1, x0:x1][::ds, ::ds]
        curv_subset = curvature[y0:y1, x0:x1][::ds, ::ds]
        elev, slp, asp, curv = da.compute(elev_subset, slope_subset, aspect_subset, curv_subset)

        print(f"   🔍 {lvl['name']}: {lvl['description']}  "
              f"win[y:{y0}:{y1}, x:{x0}:{x1}], ds={ds} → shape {elev.shape}  "
              f"in {time.time()-t0:.2f}s")

        # Global coordinates extent in km for imshow
        extent_km = [x0 * ps_km, x1 * ps_km, y0 * ps_km, y1 * ps_km]
        extracted.append(dict(
            elevation=elev, slope=slp, aspect=asp, curvature=curv,
            extent_km=extent_km, ds=ds, level=lvl
        ))

    print("\n🎨 Creating visualization panels...")

    def draw_next_extent(ax, nxt_window):
        y0n, y1n, x0n, x1n = nxt_window
        rect = Rectangle((x0n * ps_km, y0n * ps_km),
                         (x1n - x0n) * ps_km, (y1n - y0n) * ps_km,
                         fill=False, lw=2.0, edgecolor='red')
        ax.add_patch(rect)

    def style_axes(ax):
        ax.set_xlabel('x (km)')
        ax.set_ylabel('y (km)')
        ax.xaxis.set_major_locator(MaxNLocator(nbins=5))
        ax.yaxis.set_major_locator(MaxNLocator(nbins=5))

    # 4 columns per level: Elevation, Slope, Aspect, Curvature
    for i, data in enumerate(extracted):
        # Elevation + hillshade
        ax_e = plt.subplot(len(zoom_levels), 4, i*4 + 1)
        ls = LightSource(azdeg=315, altdeg=45)
        try:
            hill = ls.hillshade(data['elevation'], vert_exag=2)
            ax_e.imshow(hill, cmap='gray', alpha=0.30, origin='lower',
                        extent=data['extent_km'])
        except Exception:
            pass
        im_e = ax_e.imshow(data['elevation'], cmap='terrain', alpha=0.80, origin='lower',
                           extent=data['extent_km'])
        ax_e.set_title(f"{data['level']['name']}\n{data['level']['description']}",
                       fontsize=10, fontweight='bold')
        style_axes(ax_e)
        plt.colorbar(im_e, ax=ax_e, fraction=0.046, pad=0.04, label='Elevation (m)')

        # Slope (deg)
        ax_s = plt.subplot(len(zoom_levels), 4, i*4 + 2)
        slope_deg = np.degrees(np.arctan(data['slope']))
        im_s = ax_s.imshow(slope_deg, cmap='plasma', vmin=0, vmax=45, origin='lower',
                           extent=data['extent_km'])
        ax_s.set_title('Slope (degrees)', fontsize=10)
        style_axes(ax_s)
        plt.colorbar(im_s, ax=ax_s, fraction=0.046, pad=0.04, label='Slope (°)')

        # Aspect (deg in [0,360))
        ax_a = plt.subplot(len(zoom_levels), 4, i*4 + 3)
        aspect_deg = (np.degrees(data['aspect']) + 360.0) % 360.0
        im_a = ax_a.imshow(aspect_deg, cmap='hsv', vmin=0, vmax=360, origin='lower',
                           extent=data['extent_km'])
        ax_a.set_title('Aspect (degrees)', fontsize=10)
        style_axes(ax_a)
        plt.colorbar(im_a, ax=ax_a, fraction=0.046, pad=0.04, label='Aspect (°)')

        # Curvature (scaled)
        ax_c = plt.subplot(len(zoom_levels), 4, i*4 + 4)
        curv_scaled = data['curvature'] * 1000.0
        curv_max = np.percentile(np.abs(curv_scaled), 95)
        im_c = ax_c.imshow(curv_scaled, cmap='RdBu_r', vmin=-curv_max, vmax=curv_max,
                           origin='lower', extent=data['extent_km'])
        ax_c.set_title('Curvature (×1000 m⁻¹)', fontsize=10)
        style_axes(ax_c)
        plt.colorbar(im_c, ax=ax_c, fraction=0.046, pad=0.04, label='Curvature')

        # Red box for next level (all 4 panels)
        if i < len(extracted) - 1:
            nxt_win = extracted[i+1]['level']['window']
            for ax in (ax_e, ax_s, ax_a, ax_c):
                draw_next_extent(ax, nxt_win)

    plt.suptitle(
        f'🗻 MASSIVE TERRAIN ANALYSIS - Multi-Scale Visualization 🗻\n'
        f'Domain: {full_x_km:.0f}×{full_y_km:.0f} km  |  '
        f'Pixels: {array_size[1]}×{array_size[0]}  |  Pixel: {pixel_size:.1f} m',
        fontsize=16, fontweight='bold', y=0.98
    )
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    return fig

# === CALL ===
print("🎬 LAUNCHING SPECTACULAR MULTI-SCALE VISUALIZATION...")
viz_start = time.time()
spectacular_fig = create_multiscale_visualization(
    massive_terrain,
    terrain_metrics['slope'],
    terrain_metrics['aspect'],
    terrain_metrics['curvature'],
    ARRAY_SIZE,
    PIXEL_SIZE
)
viz_time = time.time() - viz_start
plt.show()
print(f"\n🎉 VISUALIZATION COMPLETED in {viz_time:.2f}s! 🎉")


## 📊 ULTIMATE PERFORMANCE SUMMARY

In [ ]:
print("🏆" + "═" * 60 + "🏆")
print("    🔥 DASK MASSIVE ARRAY PROCESSING - RESULTS 🔥")
print("🏆" + "═" * 60 + "🏆")

# Calculate totals
total_data_processed = 4 * np.prod(ARRAY_SIZE) * 8 / 1024**3  # 4 arrays × 8 bytes
total_computation_time = terrain_time + viz_time
final_memory = psutil.virtual_memory()

print(f"\n📏 DATASET SCALE:")
print(f"   🗺️  Geographic Coverage: {EXTENT_KM:.0f} km × {EXTENT_KM:.0f} km")
print(f"   🔢 Grid Resolution: {ARRAY_SIZE[0]:,} × {ARRAY_SIZE[1]:,} = {np.prod(ARRAY_SIZE):,} points")
print(f"   🎯 Spatial Resolution: {PIXEL_SIZE} m/pixel")
print(f"   📊 Total Points Analyzed: {np.prod(ARRAY_SIZE)/1e6:.0f} MILLION")
print(f"   💾 Raw Data Volume: ~{total_data_processed:.1f} GB")

print(f"\n⚡ COMPUTATIONAL PERFORMANCE:")
print(f"   ⏱️  Total Processing Time: {total_computation_time:.2f} seconds")
print(f"   🚀 Processing Throughput: {total_data_processed/total_computation_time:.2f} GB/s")
print(f"   🏭 Parallel Workers: {len(client.scheduler_info()['workers'])}")
print(f"   🧵 Total Threads: {sum(w['nthreads'] for w in client.scheduler_info()['workers'].values())}")
print(f"   🧮 Points/Second: {np.prod(ARRAY_SIZE)/total_computation_time/1e6:.1f} Million/s")

print(f"\n💾 MEMORY EFFICIENCY:")
print(f"   📈 Peak RAM Usage: {final_memory.used/1024**3:.1f} GB / {final_memory.total/1024**3:.1f} GB")
print(f"   🔄 Memory Efficiency: {total_data_processed/(final_memory.used/1024**3):.1f}× data-to-RAM ratio")
print(f"   ♻️  Out-of-Core: ✅ Processed {total_data_processed:.1f} GB dataset")

print(f"\n🧮 ANALYSIS ACHIEVEMENTS:")
print(f"   🏔️  Terrain Metrics: Elevation, Slope, Aspect, Curvature")
print(f"   📊 Statistical Analysis: Min/Max/Mean/Std/Percentiles")
print(f"   🎯 Classification: Steep areas, Convex/Concave regions")
print(f"   🔍 Multi-Scale Viz: 4 zoom levels (satellite → hiking detail)")
print(f"   🎨 Advanced Rendering: Hillshade, color mapping, overlays")

print(f"\n🌟 SCALE COMPARISON:")
print(f"   🌍 Area Equivalent: Larger than Belgium ({EXTENT_KM**2:.0f} km²)")
print(f"   📱 Storage: ~{total_data_processed/64:.1f}× iPhone 64GB capacity")
print(f"   ⚡ Speed: Analyzed entire country-scale terrain in {total_computation_time:.0f} seconds!")

print(f"\n🚀 DASK SUPERPOWERS DEMONSTRATED:")
print(f"   ✅ Lazy Evaluation: Build complex computation graphs")
print(f"   ✅ Parallel Processing: Automatic multi-core utilization") 
print(f"   ✅ Out-of-Core: Process datasets larger than memory")
print(f"   ✅ Chunked Operations: Efficient memory management")
print(f"   ✅ Boundary Handling: map_overlap for edge artifacts")
print(f"   ✅ Scalability: Same code → laptop to supercomputer")

print(f"\n" + "🏆" + "═" * 60 + "🏆")
print(f"   🎉 MISSION ACCOMPLISHED: MASSIVE SCALE MASTERED! 🎉")
print(f"🏆" + "═" * 60 + "🏆")

## 🧹 Cleanup and Next Steps

In [ ]:
print("🧹 Cleaning up resources...")

# Close the Dask client
client.close()
print("✅ Dask cluster shut down")

# Final memory check
final_memory = psutil.virtual_memory()
print(f"💾 Final memory usage: {final_memory.used/1024**3:.1f} GB")

print(f"\n🎓 WHAT YOU LEARNED:")
print(f"   📚 Dask fundamentals: arrays, chunks, lazy evaluation")
print(f"   ⚡ Parallel computing: multi-core processing")
print(f"   💾 Memory management: out-of-core operations")
print(f"   🧮 Advanced analytics: terrain analysis at scale")
print(f"   🎨 Data visualization: multi-scale presentations")
print(f"   🏗️  Boundary handling: map_overlap for correct gradients")

print(f"\n🚀 NEXT STEPS:")
print(f"   📈 Scale up: Try even larger datasets")
print(f"   🌐 Distributed: Use Dask on multiple machines")
print(f"   🧠 ML integration: Combine with dask-ml for machine learning")
print(f"   📊 Real data: Apply to satellite imagery, climate data")
print(f"   ☁️  Cloud deployment: Use Dask on AWS, GCP, Azure")

print(f"\n✨ You just processed {np.prod(ARRAY_SIZE)/1e6:.0f} MILLION data points! ✨")
print(f"🏆 Welcome to MASSIVE-SCALE computing with Dask! 🏆")